In [1]:
import gspread
import datetime

In [2]:
google_api_connect = gspread.service_account(filename='apisheet.json')

In [10]:
shair_file = google_api_connect.create( 'LastDataBase' )

# แชร์ไฟล์ สเปรตชีต มาให้เรา
shair_file.share(  email_address='phuphaeiei55@gmail.com' , perm_type='user' , role='writer' )

# ดึง Url สเปรตชีตออก
google_api_connect.open_by_key( shair_file.id )

<Spreadsheet 'LastDataBase' id:1Fhl6GByf_XJaqVMhHKxqVFk7NfXgYou4GoO4sEoeu-U>

In [3]:
# เชื่อมต่อ สเปรตชีต
sheet_selete = google_api_connect.open_by_key( '1-qKxl_GwyrcsHfE6h-EEvf0hdJmpOWtouoagxO-y48U' )
# เชื่อมต่อกับ Workชีต
action_sheet =  sheet_selete.worksheet( 'Sheet1' )

In [ ]:
# สร้างหัวตาราง
new_row = [ 'Date' , 'Air temperature(°C)' , 'Air humidity(%)' , 'Plant height(cm)' ]
action_sheet.insert_row( values= new_row , index=1 )

# Lock แถว
action_sheet.freeze( rows=1 , cols=1 )
action_sheet.freeze( rows=1 , cols=2 )
action_sheet.freeze( rows=1 , cols=3 )
action_sheet.freeze( rows=1 , cols=4 )

# ปรับสีตัวหนังสือสีเหลือง
table = 'A1:D1'

format_data = {

    "textFormat" : {

        "bold" : True ,

        "foregroundColor" : {

            "red" : 255 / 255,
            "green" : 255 /255,
            "blue" : 0 /255

        }


    }

}

action_sheet.format( ranges = table , format=format_data )

# ปรับสีพื้นหลังสีดำ
table = 'A1:D1'

format_data = {

    "backgroundColor" : {

        "red" : 0 /255,
        "green" : 0 /255,
        "blue" : 0 /255,
        "alpha" : 1 /255 ,

    }

}

action_sheet.format( ranges = table , format=format_data )

# จัดค่อลั่มให้อยู่ตรงกลาง
table = 'A1:D1'

format_data = {

    "horizontalAlignment" : "CENTER" ,
    "verticalAlignment" : "MIDDLE"

}

action_sheet.format( ranges = table , format=format_data )

In [5]:
def insert_data( colum_set , data_insert , in_column_selete ) :
 
    # วน Loop เพิ่มข้อมูล
    for value in data_insert :

        # ตรวจสอบว่ามีข้อมูลซ้ำแล้วหรอยัง
        if  len( action_sheet.findall( query=value[0] , in_column=in_column_selete ) ) < 1 :

            # ยังยังไม่มีข้อมูลซ้ำให้เพิ่มข้อมูล
            action_sheet.append_row(  values=value , table_range=colum_set )

# กำหนดคอลั่มที่ต้องการเพิ่มข้อมูล
colum_set = 'A:D'

# ข้อมูลที่เราต้องการเพิ่มเข้าไป
data_insert = [ [ 'Name7' , 'Password1234' , '1' , str( datetime.date.today() ) ]  ]

# คอลั่มที่ต้องทำการตรวจสอบข้อมูลซ้ำ
in_column_selete = 1

insert_data( colum_set , data_insert , in_column_selete )

In [ ]:
import gspread
import requests
import schedule
import time
from oauth2client.service_account import ServiceAccountCredentials

# การเชื่อมต่อกับ Google Sheets
google_api_connect = gspread.service_account(filename='apisheet.json')

# เชื่อมต่อกับ Spreadsheet โดยใช้ Key ของไฟล์
sheet_selete = google_api_connect.open_by_key('1-qKxl_GwyrcsHfE6h-EEvf0hdJmpOWtouoagxO-y48U')

# เชื่อมต่อกับ Worksheet ที่ต้องการ
action_sheet = sheet_selete.worksheet('Sheet1')

# ฟังก์ชันคำนวณและส่งข้อมูล Line Notify
def send_daily_report():
    data = action_sheet.get_all_values()  # ดึงข้อมูลทั้งหมดใน Worksheet

    def safe_float(value):
        try:
            return float(value)
        except ValueError:
            return None

    temperature_col = [safe_float(row[1]) for row in data[1:]]
    humidity_col = [safe_float(row[2]) for row in data[1:]]
    plant_height_col = [safe_float(row[3]) for row in data[1:]]

    temperature_col = [x for x in temperature_col if x is not None]
    humidity_col = [x for x in humidity_col if x is not None]
    plant_height_col = [x for x in plant_height_col if x is not None]

    temperature_avg = sum(temperature_col) / len(temperature_col) if temperature_col else 0
    humidity_avg = sum(humidity_col) / len(humidity_col) if humidity_col else 0
    plant_height_change = max(plant_height_col) - min(plant_height_col) if plant_height_col else 0

    report = f"""
    รายงานสรุปผลการเติบโตของพืช:
    - ค่าเฉลี่ยอุณหภูมิ: {temperature_avg:.2f}°C
    - ค่าเฉลี่ยความชื้น: {humidity_avg:.2f}%
    - การเปลี่ยนแปลงความสูงของพืช: {plant_height_change:.2f} เซนติเมตร
    """

    def send_line_notify(message):
        token = 'qHohPaduckB4TVS76ll8W8UPIBScxhjCD4Nq29BiWlC'  # ใส่ token ของ Line Notify
        url = 'https://notify-api.line.me/api/notify'
        headers = {'Authorization': 'Bearer ' + token}
        payload = {'message': message}
        response = requests.post(url, headers=headers, data=payload)
        if response.status_code == 200:
            print("ส่งข้อความ Line สำเร็จ")
        else:
            print("เกิดข้อผิดพลาดในการส่งข้อความ Line")

    send_line_notify(report)
    # หยุดการรันหลังจากส่งสำเร็จ
    return schedule.CancelJob

# ตั้งเวลาให้ฟังก์ชันรันทุกวันเวลา 12:00 น.
schedule.every().day.at("15:40").do(send_daily_report)

# ลูปรอให้ถึงเวลาเรียกฟังก์ชัน
while True:
    schedule.run_pending()
    time.sleep(20)  # ตรวจสอบทุกๆ x วินาที